# Chapter 03-05: 커스텀 학습 루프 (Custom Training Loop)

## 학습 목표
- `model.fit()`이 내부적으로 수행하는 과정을 이해한다
- `tf.GradientTape`를 사용하여 수동 학습 루프를 구현할 수 있다
- `@tf.function` 데코레이터로 그래프 모드 컴파일을 적용하여 속도를 향상시킬 수 있다
- `model.fit()`과 커스텀 루프의 장단점을 비교하고 적절한 상황을 선택할 수 있다

## 목차
1. [model.fit()이 내부적으로 하는 일](#1.-model.fit()이-내부적으로-하는-일)
2. [기본 GradientTape 학습 루프](#2.-기본-GradientTape-학습-루프)
3. [train_step / val_step 함수 분리](#3.-train_step-/-val_step-함수-분리)
4. [@tf.function으로 성능 최적화](#4.-@tf.function으로-성능-최적화)
5. [메트릭 초기화와 에포크 로그](#5.-메트릭-초기화와-에포크-로그)
6. [정리](#6.-정리)

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time

print("TensorFlow 버전:", tf.__version__)
tf.random.set_seed(42)
np.random.seed(42)

## 1. model.fit()이 내부적으로 하는 일

`model.fit()`을 호출하면 내부적으로 다음 과정이 반복된다:

1. **데이터를 배치로 분리** - `batch_size` 크기로 미니배치 생성
2. **Forward Pass (순전파)** - 모델이 예측값 계산 (`y_pred = model(X, training=True)`)
3. **Loss 계산** - 손실 함수로 예측값과 실제값 비교 (`loss = loss_fn(y_true, y_pred)`)
4. **Backward Pass (역전파)** - 자동 미분으로 그래디언트 계산 (`tape.gradient(loss, weights)`)
5. **가중치 업데이트** - 옵티마이저로 파라미터 갱신 (`optimizer.apply_gradients(...)`)
6. **메트릭 갱신** - 배치 결과를 누적하여 에포크 메트릭 계산

커스텀 루프는 이 과정을 직접 코드로 작성하여 더 세밀한 제어를 가능하게 한다.

In [ ]:
# ---------------------------------------------------
# 데이터 및 모델 준비
# ---------------------------------------------------

# MNIST 데이터 로드
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full.reshape(-1, 784).astype('float32') / 255.0
X_test = X_test.reshape(-1, 784).astype('float32') / 255.0

# 빠른 실험용 서브셋
X_train = X_train_full[:8000]
y_train = y_train_full[:8000]
X_val = X_train_full[8000:10000]
y_val = y_train_full[8000:10000]

# tf.data.Dataset으로 변환
BATCH_SIZE = 64

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.batch(BATCH_SIZE)

def build_model():
    """MLP 분류 모델 생성"""
    tf.random.set_seed(42)
    return tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)  # 로짓 출력 (활성화 없음)
    ])

print(f"훈련 배치 수: {len(train_dataset)}")
print(f"검증 배치 수: {len(val_dataset)}")

## 2. 기본 GradientTape 학습 루프

In [ ]:
# ---------------------------------------------------
# 기본 커스텀 학습 루프 구현
# ---------------------------------------------------

# 모델, 손실 함수, 옵티마이저 정의
model = build_model()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# 에포크 및 배치 수준 메트릭
train_loss_metric = tf.keras.metrics.Mean(name='train_loss')
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='train_acc')
val_loss_metric = tf.keras.metrics.Mean(name='val_loss')
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='val_acc')

EPOCHS = 3
history_custom = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

print("=== 커스텀 학습 루프 시작 ===")
total_start = time.time()

for epoch in range(EPOCHS):
    epoch_start = time.time()
    
    # -------- 훈련 루프 --------
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        
        # GradientTape 컨텍스트: 이 블록 안의 연산을 기록
        with tf.GradientTape() as tape:
            # 1. Forward pass: 예측값 계산 (training=True: Dropout 활성화)
            y_pred = model(x_batch, training=True)
            # 2. Loss 계산
            loss = loss_fn(y_batch, y_pred)
        
        # 3. Backward pass: 그래디언트 계산
        # model.trainable_variables: 학습 가능한 파라미터 (가중치 + 편향)
        gradients = tape.gradient(loss, model.trainable_variables)
        
        # 4. 가중치 업데이트
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        # 5. 메트릭 갱신
        train_loss_metric.update_state(loss)
        train_acc_metric.update_state(y_batch, y_pred)
    
    # -------- 검증 루프 --------
    for x_batch_val, y_batch_val in val_dataset:
        # 검증 시: training=False (Dropout 비활성화)
        y_pred_val = model(x_batch_val, training=False)
        val_loss = loss_fn(y_batch_val, y_pred_val)
        val_loss_metric.update_state(val_loss)
        val_acc_metric.update_state(y_batch_val, y_pred_val)
    
    # -------- 에포크 결과 출력 --------
    epoch_time = time.time() - epoch_start
    t_loss = train_loss_metric.result().numpy()
    t_acc = train_acc_metric.result().numpy()
    v_loss = val_loss_metric.result().numpy()
    v_acc = val_acc_metric.result().numpy()
    
    history_custom['train_loss'].append(t_loss)
    history_custom['train_acc'].append(t_acc)
    history_custom['val_loss'].append(v_loss)
    history_custom['val_acc'].append(v_acc)
    
    print(f"에포크 {epoch+1}/{EPOCHS} ({epoch_time:.1f}s) - "
          f"loss: {t_loss:.4f} - acc: {t_acc:.4f} - "
          f"val_loss: {v_loss:.4f} - val_acc: {v_acc:.4f}")
    
    # -------- 메트릭 초기화 (다음 에포크를 위해) --------
    train_loss_metric.reset_state()
    train_acc_metric.reset_state()
    val_loss_metric.reset_state()
    val_acc_metric.reset_state()

total_time = time.time() - total_start
print(f"\n총 학습 시간: {total_time:.1f}초")

## 3. train_step / val_step 함수 분리

배치 단위 처리를 별도 함수로 분리하면 코드 가독성이 향상되고 `@tf.function` 적용이 용이해진다.

In [ ]:
# ---------------------------------------------------
# train_step / val_step 함수로 분리
# ---------------------------------------------------

# 새 모델과 옵티마이저
model2 = build_model()
optimizer2 = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn2 = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# 메트릭 객체
tr_loss = tf.keras.metrics.Mean(name='train_loss')
tr_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='train_acc')
vl_loss = tf.keras.metrics.Mean(name='val_loss')
vl_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='val_acc')


def train_step(x_batch, y_batch):
    """단일 훈련 배치 처리: Forward -> Loss -> Backward -> Update"""
    with tf.GradientTape() as tape:
        predictions = model2(x_batch, training=True)
        loss = loss_fn2(y_batch, predictions)
    
    # 그래디언트 계산 및 가중치 업데이트
    gradients = tape.gradient(loss, model2.trainable_variables)
    optimizer2.apply_gradients(zip(gradients, model2.trainable_variables))
    
    # 메트릭 갱신
    tr_loss.update_state(loss)
    tr_acc.update_state(y_batch, predictions)


def val_step(x_batch, y_batch):
    """단일 검증 배치 처리: Forward -> Loss (그래디언트 계산 없음)"""
    # 검증 시에는 GradientTape 불필요
    predictions = model2(x_batch, training=False)
    loss = loss_fn2(y_batch, predictions)
    
    vl_loss.update_state(loss)
    vl_acc.update_state(y_batch, predictions)


def run_epoch(train_ds, val_ds):
    """전체 에포크 실행: 훈련 + 검증"""
    # 훈련
    for x_b, y_b in train_ds:
        train_step(x_b, y_b)
    
    # 검증
    for x_b, y_b in val_ds:
        val_step(x_b, y_b)
    
    # 결과 수집
    results = {
        'train_loss': tr_loss.result().numpy(),
        'train_acc': tr_acc.result().numpy(),
        'val_loss': vl_loss.result().numpy(),
        'val_acc': vl_acc.result().numpy()
    }
    
    # 메트릭 초기화
    tr_loss.reset_state()
    tr_acc.reset_state()
    vl_loss.reset_state()
    vl_acc.reset_state()
    
    return results


# 3 에포크 학습
print("=== 함수 분리 버전 학습 ===")
start = time.time()
for epoch in range(3):
    results = run_epoch(train_dataset, val_dataset)
    print(f"에포크 {epoch+1}: "
          f"loss={results['train_loss']:.4f}, "
          f"acc={results['train_acc']:.4f}, "
          f"val_loss={results['val_loss']:.4f}, "
          f"val_acc={results['val_acc']:.4f}")

elapsed_eager = time.time() - start
print(f"\n이거 모드(Eager) 소요 시간: {elapsed_eager:.1f}초")

## 4. @tf.function으로 성능 최적화

`@tf.function` 데코레이터는 Python 함수를 TensorFlow 그래프로 컴파일하여 실행 속도를 크게 향상시킨다.

- **이거 모드(Eager mode)**: Python 코드가 즉시 실행 (디버깅 편리)
- **그래프 모드(Graph mode)**: `@tf.function`으로 최적화된 정적 그래프 실행 (속도 빠름)

첫 호출 시 **트레이싱(tracing)**이 발생하여 약간 느리지만, 이후 호출은 최적화된 그래프를 사용한다.

In [ ]:
# ---------------------------------------------------
# @tf.function 적용으로 그래프 모드 컴파일
# ---------------------------------------------------

model3 = build_model()
optimizer3 = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn3 = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

tr_loss3 = tf.keras.metrics.Mean()
tr_acc3 = tf.keras.metrics.SparseCategoricalAccuracy()
vl_loss3 = tf.keras.metrics.Mean()
vl_acc3 = tf.keras.metrics.SparseCategoricalAccuracy()


@tf.function  # 이 데코레이터가 함수를 TF 그래프로 컴파일
def train_step_graph(x_batch, y_batch):
    """그래프 모드 훈련 스텝: @tf.function으로 최적화"""
    with tf.GradientTape() as tape:
        predictions = model3(x_batch, training=True)
        loss = loss_fn3(y_batch, predictions)
    gradients = tape.gradient(loss, model3.trainable_variables)
    optimizer3.apply_gradients(zip(gradients, model3.trainable_variables))
    tr_loss3.update_state(loss)
    tr_acc3.update_state(y_batch, predictions)


@tf.function
def val_step_graph(x_batch, y_batch):
    """그래프 모드 검증 스텝"""
    predictions = model3(x_batch, training=False)
    loss = loss_fn3(y_batch, predictions)
    vl_loss3.update_state(loss)
    vl_acc3.update_state(y_batch, predictions)


# 이거 모드와 그래프 모드 속도 비교
N_EPOCHS = 3

print("=== @tf.function 그래프 모드 학습 ===")
start = time.time()

for epoch in range(N_EPOCHS):
    for x_b, y_b in train_dataset:
        train_step_graph(x_b, y_b)
    for x_b, y_b in val_dataset:
        val_step_graph(x_b, y_b)
    
    print(f"에포크 {epoch+1}: "
          f"loss={tr_loss3.result().numpy():.4f}, "
          f"acc={tr_acc3.result().numpy():.4f}, "
          f"val_loss={vl_loss3.result().numpy():.4f}, "
          f"val_acc={vl_acc3.result().numpy():.4f}")
    
    tr_loss3.reset_state()
    tr_acc3.reset_state()
    vl_loss3.reset_state()
    vl_acc3.reset_state()

elapsed_graph = time.time() - start
print(f"\n그래프 모드 소요 시간: {elapsed_graph:.1f}초")

print()
print(f"이거 모드 (함수 분리 버전): {elapsed_eager:.1f}초")
print(f"그래프 모드 (@tf.function):  {elapsed_graph:.1f}초")
if elapsed_eager > 0:
    speedup = elapsed_eager / elapsed_graph
    print(f"속도 향상: {speedup:.2f}x")

print()
print("주의사항:")
print("  - @tf.function 내에서 Python print() 는 트레이싱 시에만 실행됨")
print("  - 디버깅 시에는 @tf.function을 제거하고 이거 모드로 사용")
print("  - tf.print()를 사용하면 그래프 모드에서도 값 출력 가능")

## 5. 메트릭 초기화와 에포크 로그

In [ ]:
# ---------------------------------------------------
# 완전한 커스텀 학습 루프: 메트릭 관리 + 로그 + 시각화
# ---------------------------------------------------

model4 = build_model()
optimizer4 = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn4 = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# 메트릭 딕셔너리로 관리
metrics = {
    'train_loss': tf.keras.metrics.Mean(),
    'train_acc': tf.keras.metrics.SparseCategoricalAccuracy(),
    'val_loss': tf.keras.metrics.Mean(),
    'val_acc': tf.keras.metrics.SparseCategoricalAccuracy(),
}

@tf.function
def train_step_final(x, y):
    with tf.GradientTape() as tape:
        preds = model4(x, training=True)
        loss = loss_fn4(y, preds)
    grads = tape.gradient(loss, model4.trainable_variables)
    optimizer4.apply_gradients(zip(grads, model4.trainable_variables))
    metrics['train_loss'].update_state(loss)
    metrics['train_acc'].update_state(y, preds)

@tf.function
def val_step_final(x, y):
    preds = model4(x, training=False)
    loss = loss_fn4(y, preds)
    metrics['val_loss'].update_state(loss)
    metrics['val_acc'].update_state(y, preds)

# 학습 기록
history_final = {key: [] for key in metrics.keys()}
EPOCHS_FINAL = 8

print(f"{'에포크':^8} {'Train Loss':^12} {'Train Acc':^12} {'Val Loss':^12} {'Val Acc':^12}")
print("-" * 60)

for epoch in range(EPOCHS_FINAL):
    # 훈련
    for x_b, y_b in train_dataset:
        train_step_final(x_b, y_b)
    
    # 검증
    for x_b, y_b in val_dataset:
        val_step_final(x_b, y_b)
    
    # 결과 수집 및 출력
    epoch_results = {k: v.result().numpy() for k, v in metrics.items()}
    for k, v in epoch_results.items():
        history_final[k].append(v)
    
    print(f"{epoch+1:^8d} "
          f"{epoch_results['train_loss']:^12.4f} "
          f"{epoch_results['train_acc']:^12.4f} "
          f"{epoch_results['val_loss']:^12.4f} "
          f"{epoch_results['val_acc']:^12.4f}")
    
    # 메트릭 초기화 (중요: 다음 에포크를 위해 반드시 초기화)
    for metric in metrics.values():
        metric.reset_state()

# 학습 곡선 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
epochs_range = range(1, EPOCHS_FINAL + 1)

axes[0].plot(epochs_range, history_final['train_loss'], 'b-o', label='훈련 손실', markersize=4)
axes[0].plot(epochs_range, history_final['val_loss'], 'r-o', label='검증 손실', markersize=4)
axes[0].set_xlabel('에포크')
axes[0].set_ylabel('손실')
axes[0].set_title('커스텀 루프 - 손실 곡선')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(epochs_range, history_final['train_acc'], 'b-o', label='훈련 정확도', markersize=4)
axes[1].plot(epochs_range, history_final['val_acc'], 'r-o', label='검증 정확도', markersize=4)
axes[1].set_xlabel('에포크')
axes[1].set_ylabel('정확도')
axes[1].set_title('커스텀 루프 - 정확도 곡선')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. 정리

### model.fit() vs 커스텀 학습 루프 비교

| 항목 | model.fit() | 커스텀 루프 |
|------|-------------|------------|
| **코드 양** | 적음 (간결) | 많음 (상세) |
| **유연성** | 낮음 | 높음 |
| **디버깅** | 어려움 | 쉬움 (이거 모드) |
| **콜백 지원** | 풍부한 내장 콜백 | 직접 구현 필요 |
| **분산 학습** | 내장 지원 | 추가 코드 필요 |
| **권장 상황** | 일반적인 학습 | GAN, RL, 멀티-모델 학습 등 |

### 커스텀 루프가 필요한 상황
1. **GAN (Generative Adversarial Networks)**: 생성자와 판별자를 교대로 학습
2. **강화학습**: 보상 기반 학습으로 표준 손실 함수 사용 불가
3. **멀티-태스크 학습**: 여러 손실을 다르게 처리
4. **그래디언트 클리핑/수정**: 그래디언트를 적용 전에 직접 수정
5. **모델 앙상블**: 여러 모델을 하나의 루프에서 학습

### 핵심 패턴 요약
```python
# 기본 패턴
with tf.GradientTape() as tape:
    y_pred = model(x, training=True)    # Forward pass
    loss = loss_fn(y_true, y_pred)       # Loss 계산

grads = tape.gradient(loss, model.trainable_variables)  # Backward
optimizer.apply_gradients(zip(grads, model.trainable_variables))  # Update

metric.reset_state()  # 에포크 후 반드시 초기화!
```